In [1]:
import cv2 as cv
import numpy as np
import glob
import os
import imutils
from imutils.perspective import four_point_transform
from skimage.segmentation import clear_border

In [2]:
def print_img(i,img):
    cv.imshow(str(i),img)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [3]:
def find_sudoku(i):#returns an image of just the sudoku grid from an image containing a sudoku
    image = cv.imread(images[i])
    image = cv.resize(image,(0,0),fx=0.25,fy=0.25) #resize

    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY) #to grayscale
    blurred = cv.GaussianBlur(gray, (11,11), 3) #blur

    #threshold to get grid
    thresh = cv.adaptiveThreshold(blurred, 255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2)
    thresh = cv.bitwise_not(thresh)

    # find contours in the thresholded image
    cnts = cv.findContours(thresh.copy(), cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    #sort by biggest contour first
    cnts = sorted(cnts, key=cv.contourArea, reverse=True)
    
    sudokuCnt = None
    for c in cnts:
        # approximate the contour
        peri = cv.arcLength(c, True)
        approx = cv.approxPolyDP(c, 0.02 * peri, True)
        # check to see if it has 4 points
        if len(approx) == 4:
            sudokuCnt = approx
            break
    
    # apply a four point perspective transform to original image
    #automaticaly sorts points
    sudoku = four_point_transform(image, sudokuCnt.reshape(4, 2))
    return sudoku


In [4]:
def check_digit(cell):
    # clear borders
    thresh = cv.threshold(cell, 0, 255,cv.THRESH_BINARY_INV | cv.THRESH_OTSU)[1]
    thresh = clear_border(thresh)
    
    # find contours in the thresholded cell
    cnts = cv.findContours(thresh.copy(), cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    # if no contours were found than this is an empty cell
    if len(cnts) == 0:
        return None
    
    
    # mask to check for digit
    c = max(cnts, key=cv.contourArea)
    mask = np.zeros(thresh.shape, dtype="uint8")
    cv.drawContours(mask, [c], -1, 255, -1)   
    
    # less than 2% of the mask is filled => the cell is empty
    (h, w) = thresh.shape
    percentFilled = cv.countNonZero(mask) / float(w * h)
    
    if percentFilled < 0.02:
        return None
    return 1

In [5]:
def answer(k,f):
    img = find_sudoku(k)
    print_img('a', img)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    h,w=gray.shape
    #the sizes of a cell
    h_step = int(h/9)
    w_step = int(w/9)
    #go through grid and check if cell is empty
    for i in range(9):
        for j in range(9):
            crop=gray[i*h_step:(i+1)*h_step,j*w_step:(j+1)*w_step] 
            if check_digit(crop) is None:
                f.write('o')
            else:
                f.write('x')
        if i<8:
            f.write('\n')
    f.close()

In [6]:
base_folder = "test\\classic\\" #change to "test\\classic\\" for test images
output_folder = "evaluation\\submission_files\\Partu_Ana_Maria_407\\classic\\"
images = glob.glob(os.path.join(base_folder,'*.jpg'))

for i in range(50):
    with open(output_folder+images[i][13:-4]+"_predicted.txt","w") as f:
        try:
            answer(i,f)
        except:
            pass